In [ ]:
import sqlite3
import pandas as pd


In [ ]:
query = """
    WITH ArtistGenres AS (
        SELECT 
            ag.artist_id,
            GROUP_CONCAT(ag.genre_id, ',') AS genres
        FROM 
            r_artist_genre ag
        GROUP BY 
            ag.artist_id
    ),

    NumberedArtists AS (
        SELECT 
            track_id,
            artist_id,
            ROW_NUMBER() OVER (PARTITION BY track_id) AS rn
        FROM r_track_artist
    )
    
SELECT 
    t.id, 
    a.artist_id, 
    ag.genres,
    t.duration, 
    t.popularity, 
    af.acousticness, 
    af.danceability, 
    af.energy, 
    af.instrumentalness, 
    af.key,
    af.liveness, 
    af.loudness, 
    af.speechiness, 
    af.tempo, 
    af.time_signature, 
    af.valence
FROM 
    tracks t
LEFT JOIN 
    audio_features af
ON 
    t.audio_feature_id = af.id
LEFT JOIN 
    NumberedArtists a
ON 
    t.id = a.track_id
LEFT JOIN 
    ArtistGenres ag
ON 
    a.artist_id = ag.artist_id
where a.rn = 1
"""
# LIMIT 500;

In [ ]:
conn = sqlite3.connect('/Users/idotzhori/Downloads/spotify.sqlite')
conn.text_factory = lambda b: b.decode(errors='ignore')
df = pd.read_sql_query(query, conn)

conn.close()
df.tail(50)

In [ ]:
df.to_parquet('track_info.parquet')

In [ ]:
conn = sqlite3.connect('/Users/idotzhori/Downloads/spotify.sqlite')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

table_columns_dict = {}

for table_name in tables:
    table_name = table_name[0]
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = [column[1] for column in cursor.fetchall()]
    table_columns_dict[table_name] = columns

cursor.close()
conn.close()

for table, columns in table_columns_dict.items():
    print(f"{table}: {', '.join(columns)}")


In [ ]:
select t.id, a.artist, ag.genre t.duration, t.audio_feature_id, t.popularity, af.acousticness, af.danceability, af.energy, af.instrumentalness, af.key,
af.liveness, af.loudness, af.speechiness, af.tempo, af.time_signature, af.valence
from tracks t
left join audio_features af
on t.audio_feature_id = af.id
left join r_track_artists a
on t.id = a.track_id
left join r_artist_genre ag
on a.artist_id = r_artist_genre.artist_id
